In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
# Load tickers list
stocks = pd.read_csv('sp500Stocks.csv', names=['Ticker'])
stocks

,Ticker
0,MMM
1,ABT
2,ABBV
3,ABMD
4,ACN
...,...
500,YUM
501,ZBRA
502,ZBH
503,ZION


In [3]:
#  Example data we'll use

from api import key

# Batch api call to IEX
def getBatchData(symbols, data_types):
    api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types={data_types}&token={key}'
    return requests.get(api_url).json()


# P/E ratio ~ The price-to-earnings (P/E) ratio relates a company's share price to its earnings per share.
# A high P/E ratio could mean that a company's stock is overvalued, or else that investors are expecting high growth rates in the future.
# P/B ratio ~ The P/B ratio measures the market's valuation of a company relative to its book value.
# P/S ratio ~ The price-to-sales (P/S) ratio shows how much investors are willing to pay per dollar of sales for a stock. 
# EV/EBITDA ~ It compares the value of a company, inclusive of debt and other liabilities,
# to the actual cash earnings exclusive of the non-cash expenses. 
# A lower enterprise multiple can be indicative of an undervaluation of a company.
# EV/GP ~ The EV/Gross Profit Ratio is a profitability financial ratio that estimates the enterprise value of a company to its gross profit.
# It demonstrates how many dollars of enterprise value are generated for every dollar of gross profit earned. 
# Generally, the lower the ratio, the lower is the company's net worth.

def getMetrics(data):
    values = {
        'peRatio':"P/E",
        'priceToBook':"P/B",
        'priceToSales':"P/S",
        'enterpriseValue':"EV",
    }
    
    metrics = {}
    
    for value in values.keys():
        try:
            metrics[values[value]] = data['advanced-stats'][value]
        except:
            metrics[values[value]] = np.NaN
    
    try:
        metrics["EV/GP"] = metrics['EV']/data['advanced-stats']['grossProfit']
    except:
        metrics['EV/GP'] = np.NaN
        
    try:
        metrics["EV/EBITDA"] = metrics['EV']/data['advanced-stats']['EBITDA']
    except:
        metrics['EV/EBITDA'] = np.NaN
        
    try:
        metrics["Price"] = data['quote']['latestPrice']
    except:
        metrics["Price"] = np.NaN
    
    return metrics
            

symbol = 'AAPL'
data = getBatchData(symbol,'quote,advanced-stats')
metrics = getMetrics(data[symbol])
print(metrics)

{'P/E': 28.480057228516955, 'P/B': 38.09, 'P/S': 6.99, 'EV': 2514268402653, 'EV/GP': 17.17626033051291, 'EV/EBITDA': 21.83100459329846, 'Price': 145.89}


In [4]:
from utils import batchData

stockGroupsList = batchData(stocks['Ticker'])

In [5]:
columns = ['Ticker', 'Price', 
           'P/E Ratio',
           'P/E Percentile',
           'P/B Ratio',
           'P/B Percentile',
           'P/S Ratio',
           'P/S Percentile',
           'EV/EBITDA Ratio',
           'EV/EBITDA Percentile',
           'EV/GP Ratio',
           'EV/GP Percentile',
           'RV Score'
          ]
dataDf = pd.DataFrame(columns = columns)

for stockGroups in stockGroupsList:
    
    data = getBatchData(stockGroups,'quote,advanced-stats')
            
    for symbol in stockGroups.split(','):
        
        metrics = getMetrics(data[symbol])
        
        dataDf = dataDf.append(
            pd.Series([
                symbol,
                metrics['Price'],
                metrics['P/E'],
                'N/A',
                metrics['P/B'],
                'N/A',
                metrics['P/S'],
                'N/A',
                metrics['EV/EBITDA'],
                'N/A',
                metrics['EV/GP'],
                'N/A',
                'N/A',
            ],
            index = columns),
            ignore_index=True
        )

dataDf

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,MMM,185.06,17.401129,N/A,7.14,N/A,3.06,N/A,11.961320,N/A,6.708812,N/A,N/A
1,ABT,118.70,33.810603,N/A,6.17,N/A,5.41,N/A,23.510800,N/A,9.939647,N/A,N/A
2,ABBV,111.55,30.001796,N/A,15.36,N/A,3.63,N/A,10.486474,N/A,7.447971,N/A,N/A
3,ABMD,325.23,99.131928,N/A,11.27,N/A,16.11,N/A,49.328574,N/A,18.723790,N/A,N/A
4,ACN,333.47,38.748235,N/A,11.28,N/A,4.56,N/A,22.771018,N/A,13.676173,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.62,28.326478,N/A,-4.85,N/A,5.85,N/A,21.303671,N/A,9.762850,N/A,N/A
501,ZBRA,515.34,37.527114,N/A,10.57,N/A,5.31,N/A,27.497197,N/A,11.501553,N/A,N/A
502,ZBH,152.32,35.60019,N/A,2.5,N/A,4.01,N/A,20.311748,N/A,6.697226,N/A,N/A
503,ZION,66.38,6.387334,N/A,1.37,N/A,2.93,N/A,4.954155,N/A,2.702426,N/A,N/A


In [6]:
# Show any missing data
dataDf[dataDf.isnull().any(axis=1)]

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
22,GOOG,2775.05,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
41,AON,297.15,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
64,BRK.B,282.95,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
150,DISCK,25.07,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
205,FOX,38.36,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
337,NWS,24.05,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
433,TROW,203.59,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
459,UA,18.14,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [7]:
# Drop missing data

dataDf.dropna(inplace=True)
dataDf.reset_index(inplace=True, drop=True)

dataDf[dataDf.isnull().any(axis=1)]

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score


In [10]:
from scipy.stats import percentileofscore

# Calculate the percentiles 
metricsCol = ['P/E','P/B','P/S','EV/EBITDA','EV/GP']

for metric in metricsCol:
    
    ratioCol = metric+" Ratio"
    percentileCol = metric+" Percentile"
    
    for row in dataDf.index:
        dataDf.loc[row, percentileCol] = percentileofscore(dataDf[ratioCol], dataDf.loc[row, ratioCol])
        
dataDf

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,MMM,185.06,17.401129,31.589537,7.14,71.126761,3.06,44.466801,11.961320,35.412475,6.708812,41.046278,N/A
1,ABT,118.70,33.810603,65.593561,6.17,67.002012,5.41,67.404427,23.510800,73.440644,9.939647,62.977867,N/A
2,ABBV,111.55,30.001796,61.770624,15.36,87.122736,3.63,51.810865,10.486474,27.565392,7.447971,48.289738,N/A
3,ABMD,325.23,99.131928,95.573441,11.27,81.690141,16.11,94.064386,49.328574,93.360161,18.723790,89.939638,N/A
4,ACN,333.47,38.748235,74.245473,11.28,81.891348,4.56,61.971831,22.771018,71.629779,13.676173,78.06841,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,126.62,28.326478,57.344064,-4.85,4.828974,5.85,70.020121,21.303671,68.008048,9.762850,61.770624,N/A
493,ZBRA,515.34,37.527114,71.629779,10.57,79.879276,5.31,66.197183,27.497197,80.885312,11.501553,70.82495,N/A
494,ZBH,152.32,35.60019,68.812877,2.5,33.501006,4.01,57.042254,20.311748,66.197183,6.697226,40.84507,N/A
495,ZION,66.38,6.387334,9.657948,1.37,12.77666,2.93,42.756539,4.954155,4.225352,2.702426,12.474849,N/A


In [11]:
# Calculate RV Score
from statistics import mean

metricsCol = ['P/E','P/B','P/S','EV/EBITDA','EV/GP']

for row in dataDf.index:
    percentiles = []
    for metric in metricsCol:
        percentiles.append(dataDf.loc[row, metric+' Percentile'])
    
    dataDf.loc[row, 'RV Score'] = mean(percentiles)

dataDf

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,MMM,185.06,17.401129,31.589537,7.14,71.126761,3.06,44.466801,11.961320,35.412475,6.708812,41.046278,44.72837
1,ABT,118.70,33.810603,65.593561,6.17,67.002012,5.41,67.404427,23.510800,73.440644,9.939647,62.977867,67.283702
2,ABBV,111.55,30.001796,61.770624,15.36,87.122736,3.63,51.810865,10.486474,27.565392,7.447971,48.289738,55.311871
3,ABMD,325.23,99.131928,95.573441,11.27,81.690141,16.11,94.064386,49.328574,93.360161,18.723790,89.939638,90.925553
4,ACN,333.47,38.748235,74.245473,11.28,81.891348,4.56,61.971831,22.771018,71.629779,13.676173,78.06841,73.561368
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,126.62,28.326478,57.344064,-4.85,4.828974,5.85,70.020121,21.303671,68.008048,9.762850,61.770624,52.394366
493,ZBRA,515.34,37.527114,71.629779,10.57,79.879276,5.31,66.197183,27.497197,80.885312,11.501553,70.82495,73.8833
494,ZBH,152.32,35.60019,68.812877,2.5,33.501006,4.01,57.042254,20.311748,66.197183,6.697226,40.84507,53.279678
495,ZION,66.38,6.387334,9.657948,1.37,12.77666,2.93,42.756539,4.954155,4.225352,2.702426,12.474849,16.37827


In [12]:
# Sort stocks by the highest RV Score

dataDf.sort_values('RV Score', ascending = False, inplace = True)
dataDf.reset_index(drop=True, inplace=True)

dataDf

,Ticker,Price,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,PAYC,516.78,180.845358,98.189135,40.07,96.177062,33.11,99.396378,111.744507,98.792757,37.004332,99.396378,98.390342
1,NOW,658.70,771.959496,99.597586,39.77,95.975855,25,98.792757,197.474576,99.396378,30.987334,97.384306,98.229376
2,TSLA,784.94,370.962861,99.396378,32,95.171026,18.58,95.975855,128.982820,99.195171,82.327896,100.0,97.947686
3,DXCM,557.72,103.963467,95.774648,26.79,93.762575,25.06,98.993964,121.124511,98.993964,34.254976,98.591549,97.22334
4,ENPH,155.42,116.577769,96.37827,36.14,95.573441,20.18,96.579477,78.204969,97.384306,44.464387,99.597586,97.102616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,MCK,208.89,-6.883643,6.237425,-58.82,1.609658,0.1308,0.603622,8.213024,18.309859,2.813541,12.877264,7.927565
493,L,57.68,9.488084,14.486922,0.8001,6.639839,0.9936,9.859155,5.349816,5.231388,0.963922,2.816901,7.806841
494,RE,253.01,7.700129,11.267606,0.9804,7.444668,0.9202,8.249497,6.392500,8.450704,0.893186,2.21328,7.525151
495,AAL,22.03,-2.444349,7.645875,-1.84,5.231388,0.7504,6.438632,-4.922947,2.012072,2.000208,9.255533,6.1167


In [13]:
finalDf = dataDf[:50][['Ticker','Price','P/E Ratio','P/B Ratio','P/S Ratio','EV/EBITDA Ratio','EV/GP Ratio','RV Score']]
finalDf

,Ticker,Price,P/E Ratio,P/B Ratio,P/S Ratio,EV/EBITDA Ratio,EV/GP Ratio,RV Score
0,PAYC,516.78,180.845358,40.07,33.11,111.744507,37.004332,98.390342
1,NOW,658.70,771.959496,39.77,25,197.474576,30.987334,98.229376
2,TSLA,784.94,370.962861,32,18.58,128.982820,82.327896,97.947686
3,DXCM,557.72,103.963467,26.79,25.06,121.124511,34.254976,97.22334
4,ENPH,155.42,116.577769,36.14,20.18,78.204969,44.464387,97.102616
5,MPWR,485.76,115.433558,20.9,21.79,89.894593,36.882009,96.418511
6,NVDA,207.66,73.574862,25.45,24.32,60.309525,36.500013,95.895372
7,IDXX,629.85,75.573837,72.78,17.61,56.694137,30.059866,95.694165
8,FTNT,301.00,97.970972,46.73,17.04,71.335982,19.865687,94.929577
9,TECH,508.41,141.852304,12.48,20.77,60.423029,30.931557,94.406439


In [16]:
# Save the final df to an excel file
finalDf.to_excel("raccomendedTrades.xlsx", index=False)